<a href="https://colab.research.google.com/github/youseop/group-assignment/blob/master/group_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 조 배정 (랜덤)



In [95]:
import pandas as pd
import numpy as np
import random, math
from collections import deque

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [150]:
sample = ['고재헌','강민규','김강산','김동은','김보석','김승현','김영동','김용기','김용욱','김유석','김정민','김형우','오태양','윤준영','이병호','이승민','이유섭','정우성','조성현','조영후','조정훈','최교진','최성현','허승','김나현','김홍지','송예은','조새힘','한정서','황수정']

In [98]:
student_num = 30
group_num = 10
week_num = 1

student = list(i for i in range(student_num))
random.shuffle(student)
#이름으로 저장
# for i in range(student_num):
#   student[i]=sample[student[i]]

group = np.reshape(student,(group_num,student_num//group_num))

group = pd.DataFrame(group)

path = '/content/sample_data/week_%d' %(week_num)
group.to_csv(path, header = None, index = None)

group

,0,1,2
0,9,0,29
1,17,10,15
2,1,22,11
3,14,6,20
4,3,19,18
5,25,16,2
6,7,13,8
7,21,4,28
8,12,5,24
9,23,27,26


# 조 배정 (중복 제거) - 중복되지 않고 조 배정이 불가능한 경우 중복 단계별 허용

In [162]:
######fix#######
student_num = 30
group_num = 10
######fix#######

week_num = 9

overlap = list(set() for _ in range(student_num))

#이전 조의 정보를 받아서 overlap에 함께 했던 조원들의 정보 저장
for i in range(1, week_num):
  path = path = '/content/sample_data/week_%d' %(i)
  prev_group = pd.read_csv(path, header = None)
  prev_group = prev_group.values
  for group_x in prev_group:
    for x in group_x:
      for k in group_x:
        overlap[x].add(k)

#overlap[i] - i와 함께했던 조원들
print('student 0 과 함께한 조원들 : ', overlap[0])

student 0 과 함께한 조원들 :  {0, 2, 4, 5, 6, 8, 9, 10, 13, 15, 18, 21, 22, 26, 27, 29}


In [163]:
#학생 수: 30, 그룹 수 : 11 이렇게 배수 관계가 아닌 경우를 위해 올림을 추가로 해주었다.
student_per_group = math.ceil(student_num / group_num)

print('student per group : ', student_per_group)

sample_tmp = list(i for i in range(student_num))
random.shuffle(sample_tmp)
# print(sample_tmp)

cnt_std = 0

while True:
  sample=deque(sample_tmp)
  new_group = [sample.popleft()] #새로운 그룹을 저장
  overlapped_std = set() #겹치는 학생들 저장

  for i in range(1,student_num):
    #현재 new_group은 1차원 배열이기 때문에 같은 조만 탐색해야한다.
    should_check = i % student_per_group
    overlapped_tmp = set() #겹치는 학생들 임시저장

    for _ in range(len(sample)):
      cnt = 0

      #그룹 내에서 겹치는 사람이 있는지 탐색
      for k in range(1,1+should_check):
        if set([new_group[i-k]]) & overlap[sample[0]]: 
          cnt += 1
          sample.append(sample.popleft())

        if cnt > cnt_std: 
          break

      else:   
        new_group.append(sample.popleft())   
        if cnt != 0:
          overlapped_tmp.add(new_group[-1])
        break
    
    else:
      overlapped_std |= overlapped_tmp
      cnt_std += 1
      break 

  if student_num == len(new_group):
    break

print('new group : ',new_group)
print('overlapped student : ',overlapped_std)

print('최대로 겹치는 사람의 수 : ', cnt_std)

####################display as a name##################################
sample = ['고재헌','강민규','김강산','김동은','김보석','김승현','김영동','김용기','김용욱','김유석','김정민','김형우','오태양','윤준영','이병호','이승민','이유섭','정우성','조성현','조영후','조정훈','최교진','최성현','허승','김나현','김홍지','송예은','조새힘','한정서','황수정']
for i in range(student_num):
  if i%student_per_group == 0: print()
  print(sample[new_group[i]], end = ' ')
####################display as a name##################################

student per group :  3
[12, 14, 25, 23, 15, 3, 11, 18, 22, 24, 6, 17, 0, 1, 28, 20, 29, 16, 27, 10, 2, 9, 7, 4, 13, 19, 21, 5, 26, 8]
new group :  [12, 14, 15, 3, 11, 22, 24, 17, 0, 1, 20, 29, 16, 27, 10, 2, 9, 4, 13, 19, 5, 26, 25, 18, 6, 7, 21, 8, 28, 23]
overlapped student :  set()
최대로 겹치는 사람의 수 :  2

오태양 이병호 이승민 
김동은 김형우 최성현 
김나현 정우성 고재헌 
강민규 조정훈 황수정 
이유섭 조새힘 김정민 
김강산 김유석 김보석 
윤준영 조영후 김승현 
송예은 김홍지 조성현 
김영동 김용기 최교진 
김용욱 한정서 허승 

In [164]:
#1차원 배열이었던 new_group을 10 X 3 배열로 reshape
new_group = np.reshape(new_group,(group_num,student_per_group))
#numpy(array)형태의 배열을 DataFrame으로 변경(csv파일로 저장하기 위해)
new_group = pd.DataFrame(new_group)

#저장 경로 지정 - gdrive 연동 후 경로에 맞게 수정 필요
path = '/content/sample_data/week_%d' %(week_num)
new_group.to_csv(path, header = None, index = None)

print('\n %d 조 배치 완료!' %(week_num))


 9 조 배치 완료!


# 실제로 안겹치는지 check - (미완)



In [158]:
week_num = 7

overlap = list(set() for _ in range(student_num))
overlapped_std = set()

#이전 조의 정보를 받아서 overlap에 함께 했던 조원들의 정보 저장
for i in range(1, week_num+1):
  path = path = '/content/sample_data/week_%d' %(i)
  prev_group = pd.read_csv(path, header = None)
  prev_group = prev_group.values

  for group_x in prev_group:
    for x in group_x:
      for k in group_x:
        if k not in overlap[x]:
          overlap[x].add(k)
        elif k != x:
          overlapped_std.add((x,k))
          
overlapped_std

{(5, 8),
 (7, 28),
 (8, 5),
 (9, 23),
 (10, 17),
 (11, 26),
 (17, 10),
 (21, 22),
 (22, 21),
 (23, 9),
 (26, 11),
 (27, 29),
 (28, 7),
 (29, 27)}